<a href="https://colab.research.google.com/github/JoaoAcq/Agente_Culinario_IA/blob/main/agenteCulinario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q google-generativeai
!pip install -q google-adk
!pip install -q --upgrade google-generativeai

import google.generativeai as genai
from google.colab import userdata
import os

try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    print("API Key do Gemini configurada com sucesso!")
except Exception as e:
    print(f"Erro ao configurar a API Key: {e}")
    print("Verifique se você configurou o segredo 'GOOGLE_API_KEY' corretamente no Colab.")

In [11]:
# (CÉLULA 2)

import os
import textwrap
import datetime
from IPython.display import display, Markdown, HTML
from google.colab import userdata
import re

def extrair_nome_receita(texto_receita):
    """Extrai o nome da receita de um bloco de texto formatado pelo Chef."""
    match = re.search(r"---\s*RECEITA\s*\d+:\s*(.*?)\s*---", texto_receita)
    if match:
        return match.group(1).strip()
    return "Receita Desconhecida"

def extrair_ingredientes_chave(texto_receita):
    """Extrai os ingredientes chave de um bloco de texto formatado pelo Chef."""
    match = re.search(r"INGREDIENTES CHAVE PARA ESTA RECEITA:\s*(.*)", texto_receita, re.IGNORECASE)
    if match:
        ingredientes_str = match.group(1).strip()
        fim_match = re.search(r"### FIM DA RECEITA", ingredientes_str, re.IGNORECASE)
        if fim_match:
            ingredientes_str = ingredientes_str[:fim_match.start()].strip()
        return [ing.strip() for ing in ingredientes_str.split(',') if ing.strip()]
    return []

def parse_multiplas_receitas_chef(resposta_chef_completa):
    """
    Divide a resposta completa do Chef em blocos de receita individuais.
    Retorna uma lista de dicionários, cada um com 'nome', 'texto_completo' e 'ingredientes_chave'.
    """
    receitas_parseadas = []
    blocos_receita = re.split(r"###\s*Fim da Receita\s*\d+\s*###", resposta_chef_completa)

    contador_receita = 1
    for bloco in blocos_receita:
        bloco_limpo = bloco.strip()
        if not bloco_limpo:
            continue

        if re.search(r"---\s*RECEITA\s*\d+:", bloco_limpo):
            nome = extrair_nome_receita(bloco_limpo)
            ing_chave = extrair_ingredientes_chave(bloco_limpo)
            receitas_parseadas.append({
                "numero": contador_receita,
                "nome": nome,
                "texto_completo": bloco_limpo,
                "ingredientes_chave": ing_chave
            })
            contador_receita += 1

    if not receitas_parseadas and "--- RECEITA 1:" in resposta_chef_completa :
         nome = extrair_nome_receita(resposta_chef_completa)
         ing_chave = extrair_ingredientes_chave(resposta_chef_completa)
         receitas_parseadas.append({
             "numero": 1,
             "nome": nome,
             "texto_completo": resposta_chef_completa,
             "ingredientes_chave": ing_chave
         })

    return receitas_parseadas

genai_module_successful = False
try:
    from google import genai as genai_from_google_base
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
    if hasattr(genai_from_google_base, 'configure'):
        try:
            genai_from_google_base.configure(api_key=GOOGLE_API_KEY)
        except Exception as e_configure:
            print(f"Aviso: Falha ao chamar genai_from_google_base.configure(): {e_configure}")
            print("Continuando, pois a API key na variável de ambiente pode ser suficiente.")
    client = genai_from_google_base.Client()
    genai_module_successful = True

except ImportError as e_imp:
    print(f"ERRO DE IMPORTAÇÃO: Falha ao tentar 'from google import genai'. Erro: {e_imp}")
    print("Isso significa que o setup do 'Agentes_Alura.ipynb' não é diretamente replicável com esta importação.")
    print("Vamos tentar o fallback para 'google.generativeai' (que estava dando erro de Client).")
    client = None
except AttributeError as e_attr:
    print(f"ERRO DE ATRIBUTO com 'from google import genai': {e_attr}")
    print("O 'genai' importado de 'from google import genai' não tem 'Client' ou 'configure' como esperado.")
    print("Verifique os pacotes exatos usados no 'Agentes_Alura.ipynb'.")
    client = None
except Exception as e_geral:
    print(f"ERRO GERAL na tentativa de seguir 'Agentes_Alura.ipynb': {e_geral}")
    client = None

if not genai_module_successful:
    try:
        import google.generativeai as genai_google_generativeai
        GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
        genai_google_generativeai.configure(api_key=GOOGLE_API_KEY)
        # Este é o ponto que falhava antes
        client = genai_google_generativeai.Client()
        genai_module_to_use = genai_google_generativeai # Para a função abaixo
    except AttributeError as e_attr_fallback:
        print(f"ERRO DE ATRIBUTO no Fallback: {e_attr_fallback} (genai_google_generativeai não tem Client)")
        print("Isso confirma o problema anterior com a biblioteca google-generativeai.")
        print("Atributos disponíveis no 'genai_google_generativeai':")
        if 'genai_google_generativeai' in globals():
             for attr_name in dir(genai_google_generativeai): print(attr_name)
        client = None
        genai_module_to_use = None
    except Exception as e_fallback:
        print(f"ERRO GERAL no Fallback: {e_fallback}")
        client = None
        genai_module_to_use = None
else:
    genai_module_to_use = genai_from_google_base

MODEL_ID_REF = "gemini-2.0-flash"

generation_config_api = {
    "temperature": 0.7,
    }

def exibir_texto_markdown(texto):
    display(Markdown(textwrap.indent(texto, '> ', predicate=lambda _: True)))

def gerar_resposta_gemini(prompt_usuario, instrucao_sistema=None, usar_busca_google=True):
    if not client:
        return "ERRO CRÍTICO: Cliente ('client') não foi inicializado com sucesso. Verifique as mensagens acima."
    if not hasattr(client, 'models') or not hasattr(client.models, 'generate_content'):
        print(f"ERRO: O objeto 'client' (tipo: {type(client)}) não tem 'client.models.generate_content' como esperado pelo 'Agentes_Alura.ipynb'.")
        print("Atributos do client:", dir(client))
        if hasattr(client, 'models'):
            print("Atributos de client.models:", dir(client.models))
        return "ERRO CRÍTICO: Estrutura do cliente incompatível com 'Agentes_Alura.ipynb'."

    try:
        conteudo_final_para_api = prompt_usuario
        if instrucao_sistema:
            conteudo_final_para_api = f"Instrução de Sistema: {instrucao_sistema}\n\nUsuário: {prompt_usuario}"
        config_chamada_api = {"generation_config": generation_config_api}

        if usar_busca_google:

            config_chamada_api={"tools":[{"google_search":{}}]}
            print("   🔎 Usando busca do Google...")

        response = client.models.generate_content(
            model=MODEL_ID_REF,
            contents=conteudo_final_para_api,
            config=config_chamada_api
        )

        texto_resposta = ""
        if hasattr(response, 'text') and response.text:
            texto_resposta = response.text
        elif response.candidates and response.candidates[0].content.parts:
            texto_resposta = "".join(part.text for part in response.candidates[0].content.parts if hasattr(part, 'text'))


        if usar_busca_google and response.candidates and hasattr(response.candidates[0], 'grounding_metadata') and response.candidates[0].grounding_metadata:
            gm_data = response.candidates[0].grounding_metadata
            if hasattr(gm_data, 'web_search_queries') and gm_data.web_search_queries:
                print("\n   🔎 Informações da busca realizada pelo Gemini:")
                print(f"      Termos buscados: {gm_data.web_search_queries}")
            if hasattr(gm_data, 'search_entry_point') and gm_data.search_entry_point and hasattr(gm_data.search_entry_point, 'rendered_content'):
                print("      Visualização da busca (HTML):")
                display(HTML(gm_data.search_entry_point.rendered_content))
            elif hasattr(gm_data, 'grounding_chunks') and gm_data.grounding_chunks:
                print(f"      Fontes consideradas:")
                for chunk in gm_data.grounding_chunks:
                     if hasattr(chunk, 'web') and hasattr(chunk.web, 'title') and hasattr(chunk.web, 'uri'):
                        print(f"        - Título: {chunk.web.title}, URI: {chunk.web.uri}")

        if not texto_resposta:
            if response.candidates and response.candidates[0].finish_reason == "SAFETY":
                safety_ratings_info = str(response.candidates[0].safety_ratings)
                return f"Desculpe, não posso gerar uma resposta para isso. A solicitação foi bloqueada por motivos de segurança. Classificações: {safety_ratings_info}"
            return "Desculpe, não consegui gerar uma resposta válida ou a resposta estava vazia."

        return texto_resposta

    except Exception as e:
        return f"⚠️ Desculpe, ocorreu um erro inesperado ao gerar a resposta (API Client Models): {type(e).__name__} - {e}."

--- Tentando configurar API e Cliente como no 'Agentes_Alura.ipynb' ---
SUCESSO: Importado 'genai' de 'from google import genai'.
API Key definida na variável de ambiente GOOGLE_API_KEY.
SUCESSO: 'client = genai_from_google_base.Client()' criado. Tipo: <class 'google.genai.client.Client'>
Célula 2 REVISADA para tentar seguir o padrão exato do 'Agentes_Alura.ipynb'. Execute-a.


In [ ]:
def agente_chef_criativo():
    """Função principal para interagir com o Agente Chef Criativo de Geladeira."""

    exibir_texto_markdown(
        "Olá! Sou o **Chef Criativo de Geladeira**! 🍳✨\n"
        "Estou aqui para transformar seus ingredientes em obras-primas culinárias!\n"
        "Não tenha medo de ousar, a cozinha é o nosso laboratório de sabores!"
    )


    instrucao_sistema_chef = (
        "Você é o 'Chef Criativo de Geladeira', um assistente de culinária virtual com uma personalidade vibrante, "
        "extremamente criativa e um pouco excêntrica. Você adora inventar pratos com nomes divertidos e combinações inusitadas, "
        "mas sempre saborosas. Seu principal objetivo é ajudar o usuário a utilizar os ingredientes que ele já possui em casa, "
        "evitando o desperdício e incentivando a experimentação. \n\n"
        "Regras para suas respostas:\n"
        "1. Apresente-se brevemente de forma animada no início da primeira sugestão.\n"
        "2. Ao receber os ingredientes, agradeça de forma entusiástica.\n"
        "3. Sugira 1 ou 2 receitas criativas. Dê nomes únicos e divertidos para cada receita.\n"
        "4. Para cada receita, liste claramente os 'Ingredientes da Receita' (usando apenas o que o usuário forneceu ou ingredientes muito básicos como sal, pimenta, azeite, que podem ser presumidos) "
        "e o 'Modo de Preparo' em passos fáceis de seguir.\n"
        "5. Use uma linguagem encorajadora e divertida. Emojis são bem-vindos! 🎉🍲\n"
        "6. Se o usuário listar poucos ingredientes, você pode sugerir adicionar algum item básico (opcionalmente) para enriquecer o prato, mas foque no que foi dado.\n"
        "7. Não faça perguntas sobre restrições alimentares a menos que o usuário mencione algo. Se ele mencionar, adapte a receita criativamente.\n"
        "8. Ao final das sugestões, pergunte se o usuário quer tentar outra combinação de ingredientes ou se está satisfeito."
        "\n\n**INSTRUÇÕES IMPORTANTES PARA FORMATAÇÃO DA SUA RESPOSTA (SEMPRE SIGA ISSO):**\n"
        "1. Se você sugerir MAIS DE UMA receita, comece CADA receita com '--- RECEITA [NÚMERO]: [NOME DA RECEITA] ---' (ex: '--- RECEITA 1: Torta Estelar de Frango ---').\n"
        "2. Após os detalhes de CADA receita (ingredientes e modo de preparo), inclua uma seção EXATAMENTE assim: 'INGREDIENTES CHAVE PARA ESTA RECEITA: [lista dos 3 a 5 ingredientes mais importantes separados por vírgula]'.\n"
        "3. Termine CADA receita com um delimitador claro: '### Fim da Receita [NÚMERO] ###'."
        "Se for apenas UMA receita, use o mesmo formato (começando com '--- RECEITA 1: ...' e terminando com '### FIM DA RECEITA 1 ###' e incluindo 'INGREDIENTES CHAVE...')."
    )

    while True:
        ingredientes_input = input("\n👉 Por favor, me diga quais ingredientes você tem na sua geladeira e despensa hoje (separe por vírgula ou descreva): ")

        if not ingredientes_input.strip():
            exibir_texto_markdown("🌶️ Ops! Parece que você não me disse nenhum ingrediente. Vamos tentar de novo!")
            continue

        if ingredientes_input.lower() in ['sair', 'voltar', 'menu', 'fim', 'parar']:
            exibir_texto_markdown("👋 Entendido! Espero que tenha se inspirado. Até a próxima aventura na cozinha! Voltando ao menu...")
            break

        prompt_chef = f"Considerando os ingredientes que tenho: {ingredientes_input}. Me surpreenda com suas ideias!"

        exibir_texto_markdown("\n🍳 Maravilha! Ingredientes recebidos! Deixe-me aquecer minhas ideias e ver que mágica podemos fazer... ✨\n---")

        resposta_formatada = gerar_resposta_gemini(prompt_chef, instrucao_sistema=instrucao_sistema_chef)


        exibir_texto_markdown(resposta_formatada)
        exibir_texto_markdown("\n---\nEspero que essas ideias tenham aberto seu apetite! 😋")


        exibir_texto_markdown(resposta_formatada)
        exibir_texto_markdown("\n---\nEspero que essas ideias tenham aberto seu apetite! 😋")

        receitas_sugeridas = parse_multiplas_receitas_chef(resposta_formatada)
        receita_selecionada_obj = None

        if not receitas_sugeridas:
            exibir_texto_markdown("😕 Parece que não consegui extrair receitas individuais da sugestão do Chef desta vez.")
        elif len(receitas_sugeridas) == 1:
            receita_selecionada_obj = receitas_sugeridas[0]
            exibir_texto_markdown(f"O Chef sugeriu: **{receita_selecionada_obj['nome']}**.")
        else:
            exibir_texto_markdown("\nO Chef sugeriu algumas receitas! Qual delas te interessa mais para o próximo passo?")
            for i, r in enumerate(receitas_sugeridas):
                print(f"{i+1}. {r['nome']}")

            while True:
                try:
                    escolha_num = int(input(f"Digite o número da receita (1-{len(receitas_sugeridas)}) ou 0 para nenhuma: "))
                    if 0 <= escolha_num <= len(receitas_sugeridas):
                        if escolha_num > 0:
                            receita_selecionada_obj = receitas_sugeridas[escolha_num - 1]
                        break
                    else:
                        print("Número inválido, tente novamente.")
                except ValueError:
                    print("Por favor, digite um número.")


        if receita_selecionada_obj:
            nome_receita_para_prompt = receita_selecionada_obj['nome']
            print(f"\nPara a receita '{nome_receita_para_prompt}', o que você gostaria de fazer agora?")
        else:
            nome_receita_para_prompt = "as sugestões do Chef"
            print(f"\nConsiderando {nome_receita_para_prompt}, o que você gostaria de fazer agora?")

        print("1. 💰 Buscar ofertas para ingredientes? (Caçador de Ofertas)")
        print("2. 🥗 Receber dicas nutricionais ou substituições? (Nutricionista Lúdico)")
        print("3. 🧑‍🍳 Nova consulta com o Chef Criativo (outros ingredientes)")
        print("4. 🚪 Voltar ao Menu Principal do sistema")

        escolha_proximo_passo = input("Digite sua escolha (1-4): ").strip()

        if escolha_proximo_passo == '1':
            if receita_selecionada_obj and receita_selecionada_obj['ingredientes_chave']:
                exibir_texto_markdown(f"\n--- Enviando ingredientes de '{nome_receita_para_prompt}' para o Caçador de Ofertas! 🛒 ---\n")
                agente_cacador_de_ofertas(ingredientes_da_receita=receita_selecionada_obj['ingredientes_chave'])
            else:

                ingredientes_para_oferta_str = input("Quais ingredientes você gostaria de procurar ofertas? (separe por vírgula): ").strip()
                if ingredientes_para_oferta_str:
                    lista_ingredientes_para_buscar = [ing.strip() for ing in ingredientes_para_oferta_str.split(',')]
                    if lista_ingredientes_para_buscar:
                        exibir_texto_markdown("\n--- Enviando para o Caçador de Ofertas! 🛒 ---\n")
                        agente_cacador_de_ofertas(ingredientes_da_receita=lista_ingredientes_para_buscar)
            exibir_texto_markdown("\n--- Retornando ao Chef Criativo! ---")

        elif escolha_proximo_passo == '2':
            if receita_selecionada_obj and receita_selecionada_obj['texto_completo']:
                exibir_texto_markdown(f"\n--- Enviando a receita '{nome_receita_para_prompt}' para o Nutri Coach! 🥕 ---\n")
                agente_nutricionista_ludico(receita_para_analise=receita_selecionada_obj['texto_completo'])
            else:

                exibir_texto_markdown("Não consegui identificar uma receita específica para o Nutricionista.")
                analisar_mesmo_assim = input("Deseja que o Nutricionista analise o bloco de sugestões original do Chef? (sim/não): ").lower()
                if analisar_mesmo_assim == 'sim':
                     agente_nutricionista_ludico(receita_para_analise=resposta_formatada)
            exibir_texto_markdown("\n--- Retornando ao Chef Criativo! ---")


        elif escolha_proximo_passo == '3':
            exibir_texto_markdown("\nOk! Vamos para uma nova rodada com o Chef Criativo!")

            continue

        elif escolha_proximo_passo == '4':
            exibir_texto_markdown("Certo! Retornando ao menu principal do sistema.")
            return

        else:
            exibir_texto_markdown("Opção inválida. Vamos tentar uma nova consulta com o Chef.")
            continue
    return

In [12]:
def agente_cacador_de_ofertas(ingredientes_da_receita=None):
    """Função principal para interagir com o Agente Caçador de Ofertas."""

    exibir_texto_markdown(
        "Olá! Sou o **Caçador de Ofertas**! 💰🛒\n"
        "Farejando as melhores promoções para encher seu carrinho e economizar seu dinheirinho!\n"
        "O que vamos caçar hoje?"
    )

    instrucao_sistema_cacador = (
        "Você é o 'Caçador de Ofertas', um assistente de compras super eficiente, animado e com faro para bons negócios. "
        "Sua missão é usar a ferramenta de busca do Google para encontrar ofertas e promoções REAIS e ATUAIS para os ingredientes ou produtos alimentícios que o usuário solicitar. "
        "Sempre que possível, mencione a loja ou fonte da oferta e a data/validade, se a busca fornecer essa informação. "
        "Seja direto e prático nas suas respostas. Use exclamações e emojis para manter o tom divertido e de 'caçada'.\n"
        "Se o usuário perguntar sobre uma localidade específica, use-a na busca. Caso contrário, pode assumir buscas mais gerais ou para 'Guarulhos, SP' (data atual: 17 de maio de 2025), mas sempre priorize a localidade se o usuário informar.\n"
        "Se não encontrar uma oferta clara, informe que 'a caçada de hoje para este item não deu frutos 🙁, mas podemos tentar outros!'.\n"
        "Lembre o usuário de sempre confirmar as ofertas diretamente com as lojas, pois os preços e estoques mudam rápido."
    )


    if ingredientes_da_receita:
        ingredientes_str = ", ".join(ingredientes_da_receita)
        exibir_texto_markdown(f"\n🚨 Alerta do Chef! Temos uma receita fresquinha com: **{ingredientes_str}**!")

        confirmacao = input(f"Deseja que eu fareje ofertas para estes ingredientes? (sim/não): ").lower()
        if confirmacao == 'sim':
            prompt_para_gemini = f"Caçador, encontre as melhores ofertas atuais para os seguintes ingredientes de uma receita: {ingredientes_str}. Priorize ofertas em Guarulhos, SP, se possível, ou gerais. Mencione lojas/fontes se encontrar."
            exibir_texto_markdown("\n🕵️ Ok! Iniciando a caçada para os ingredientes da receita...\n---")
            resposta_busca = gerar_resposta_gemini(prompt_para_gemini, instrucao_sistema=instrucao_sistema_cacador, usar_busca_google=True)
            exibir_texto_markdown(resposta_busca)
            exibir_texto_markdown("\n---\nEspero que essas ofertas ajudem no seu banquete econômico! 🍖🥕")
        else:
            exibir_texto_markdown("Entendido! Deixaremos esses ingredientes de lado por enquanto.")


    while True:
        print("\n" + "-"*30)
        item_busca = input("👉 Qual item você quer que eu 'cace' ofertas hoje? (ou digite 'menu' para voltar): ")

        if not item_busca.strip():
            exibir_texto_markdown("Você precisa me dizer o que caçar! 😉")
            continue
        if item_busca.lower() in ['menu', 'sair', 'voltar']:
            exibir_texto_markdown("Até a próxima caçada! Boas economias! 👋")
            break

        local_busca = input(f"Quer que eu procure ofertas para '{item_busca}' em alguma cidade/região específica? (Enter para busca geral/Guarulhos): ").strip()

        prompt_usuario_cacador = f"Caçador, encontre ofertas para '{item_busca}'."

        data_atual_obj = datetime.date.today()
        meses_pt = ["janeiro", "fevereiro", "março", "abril", "maio", "junho",
                "julho", "agosto", "setembro", "outubro", "novembro", "dezembro"]
        data_formatada = f"{data_atual_obj.day} de {meses_pt[data_atual_obj.month - 1]} de {data_atual_obj.year}"

        if local_busca:
            prompt_usuario_cacador += f" em '{local_busca}'. Hoje é {data_formatada}"
        else:
            prompt_usuario_cacador += f" (pode ser busca geral ou focada em Guarulhos, SP - hoje é {data_formatada})."
        prompt_usuario_cacador += " Apresente as melhores descobertas!"

        exibir_texto_markdown("\n🕵️ Entendido! Preparando minhas ferramentas de caça...\n---")
        resposta_cacador = gerar_resposta_gemini(prompt_usuario_cacador, instrucao_sistema=instrucao_sistema_cacador, usar_busca_google=True)
        exibir_texto_markdown(resposta_cacador)
        exibir_texto_markdown("\n---\nEspero ter encontrado um tesouro para você! 💰")

        continuar = input("Quer caçar mais alguma coisa? (sim/menu): ").lower()
        if continuar != 'sim':
            exibir_texto_markdown("Certo! Voltando ao menu principal.")
            break
    return

Função 'agente_cacador_de_ofertas' definida com sucesso!


In [10]:
def agente_nutricionista_ludico(receita_para_analise=None):
    """Função principal para interagir com o Agente Nutricionista Lúdico e Motivador de Hábitos."""

    try:
        meses_pt = ["janeiro", "fevereiro", "março", "abril", "maio", "junho",
                    "julho", "agosto", "setembro", "outubro", "novembro", "dezembro"]
        data_atual_obj = datetime.date.today()
        data_formatada_atual = f"{data_atual_obj.day} de {meses_pt[data_atual_obj.month - 1]} de {data_atual_obj.year}"
    except Exception:
        data_formatada_atual = "uma data recente"
        print("AVISO: Não foi possível obter a data atual dinamicamente para o Nutricionista.")


    instrucao_sistema_nutri_adaptada = (
        f"Você é o 'Nutri Coach Divertido', um especialista em bem-estar e nutrição com uma abordagem extremamente positiva, leve, motivadora e divertida. "
        f"Seu principal objetivo é educar e inspirar os usuários a adotarem hábitos alimentares mais saudáveis e um estilo de vida equilibrado, sem usar linguagem técnica complexa ou fazer prescrições. "
        f"NÃO FORNEÇA CONSELHOS MÉDICOS OU PLANOS DE DIETA ESPECÍFICOS. Em vez disso, ofereça informações gerais, curiosidades (buscadas no Google quando apropriado), sugestões de substituições inteligentes e proponha pequenos desafios lúdicos e alcançáveis. "
        f"Use uma linguagem vibrante, muitos emojis positivos (como 🍓, 🥦, 💪, ✨, 🎉) e trate o usuário como um amigo que você está incentivando. "
        f"A data atual é {data_formatada_atual}. Você pode usar 'Guarulhos, SP' como localidade de referência se for relevante para alguma informação geral.\n"
        f"Quando o usuário pedir um 'desafio', sugira algo como 'beber 2 litros de água hoje', 'experimentar um vegetal novo esta semana', etc.\n"
        f"IMPORTANTE: Se o prompt do usuário incluir o texto de uma receita para análise (geralmente iniciada por 'Analise esta receita:'), seu foco deve ser:\n"
        f"1. Fornecer de 2 a 3 informações nutricionais gerais e interessantes sobre os ingredientes principais da receita (usando a busca do Google se necessário).\n"
        f"2. OU, se o usuário pedir especificamente, sugerir 1 ou 2 substituições saudáveis e criativas para ingredientes da receita, explicando os benefícios de forma animada.\n"
        f"Mantenha o tom divertido e motivador em todas as interações."
    )

    if not receita_para_analise:
        exibir_texto_markdown(
            "E aí, campeão(ã) da saúde! Sou seu **Nutri Coach Divertido**! 🤸‍♀️🍏\n"
            "Pronto(a) para uma dose de bem-estar com muitas dicas alto astral, curiosidades e desafios que vão te fazer sorrir de orelha a orelha?\n"
            "Me conta, qual a boa de hoje? Quer falar sobre algum alimento, melhorar um hábito ou encarar um desafio divertido?"
        )
    else:
        exibir_texto_markdown(f"\n✨ O Chef Criativo me passou uma receita deliciosa e me pediu para dar um toque nutritivo nela! A receita é:\n```\n{receita_para_analise}\n```")

        acao_nutri = input("Você gostaria de: \n1. Informações nutricionais gerais sobre esta receita? \n2. Sugestões de substituições saudáveis? \n(Digite 1, 2 ou 'não' para pular): ").lower()

        prompt_para_analise = ""
        if acao_nutri == '1':
            prompt_para_analise = f"Analise esta receita:\n```\n{receita_para_analise}\n```\nPor favor, forneça de 2 a 3 informações nutricionais gerais e interessantes sobre os ingredientes principais, de forma divertida e motivadora. Use a busca do Google se necessário."
        elif acao_nutri == '2':
            prompt_para_analise = f"Analise esta receita:\n```\n{receita_para_analise}\n```\nPor favor, sugira 1 ou 2 substituições criativas e saudáveis para ingredientes desta receita, explicando os benefícios de forma animada. Use a busca do Google se necessário."

        if prompt_para_analise:
            exibir_texto_markdown("\n💡 Perfeito! Vamos ver como podemos deixar essa delícia ainda mais poderosa e saudável...\n---")
            resposta_analise = gerar_resposta_gemini(prompt_para_analise, instrucao_sistema=instrucao_sistema_nutri_adaptada, usar_busca_google=True)
            exibir_texto_markdown(resposta_analise)
            exibir_texto_markdown("\n---\nEspero que essa análise tenha te inspirado! 🌿")
        else:
            exibir_texto_markdown("Ok, sem análise da receita por agora!")

    while True:
        print("\n" + "-"*30)

        if receita_para_analise:
             solicitacao_usuario = input("👉 Tem mais alguma dúvida sobre nutrição, quer um desafio ou posso ajudar em algo mais? (ou 'menu'): ").lower()
        else:
             solicitacao_usuario = input("👉 Sobre qual alimento, refeição, ou hábito saudável você gostaria de dicas ou curiosidades? (ou 'desafio', ou 'menu'): ").lower()


        if not solicitacao_usuario.strip():
            exibir_texto_markdown("Vamos lá, me diga como posso te ajudar a brilhar hoje! ✨")
            continue
        if solicitacao_usuario in ['menu', 'sair', 'voltar']:
            exibir_texto_markdown("Continue trilhando esse caminho de saúde e alegria! Até a próxima! 👋")
            break

        prompt_para_gemini = ""
        if "desafio" in solicitacao_usuario:
            prompt_para_gemini = (
                "Nutri Coach, estou pronto(a) para um desafio! "
                "Me sugira um desafio de alimentação ou hábito saudável que seja pequeno, divertido, fácil de começar e que traga um impacto positivo. "
                "Explique o porquê desse desafio ser uma boa e como posso realizá-lo. Seja criativo e motivador!"
            )
        else:
            prompt_para_gemini = (
                f"Nutri Coach Divertido, gostaria de informações, dicas ou curiosidades sobre: '{solicitacao_usuario}'. "
                "Quero aprender de forma leve, prática e baseada em informações confiáveis (pode usar o Google para isso, se achar útil!). "
                "Lembre-se de manter o tom super positivo e motivador!"
            )

        exibir_texto_markdown("\n💡 Ótima escolha! Deixa eu consultar meus superpoderes nutritivos (e o Google 😉)...\n---")
        resposta_nutri = gerar_resposta_gemini(prompt_para_gemini, instrucao_sistema=instrucao_sistema_nutri_adaptada, usar_busca_google=True)
        exibir_texto_markdown(resposta_nutri)
        exibir_texto_markdown("\n---\nEspero que essa dica te encha de energia e boas vibrações! 🌟")

        continuar = input("Quer mais alguma dica alto astral ou outro desafio? (sim/menu): ").lower()
        if continuar != 'sim':
            exibir_texto_markdown("Combinado! Lembre-se: cada passo saudável é uma vitória! Voltando ao menu principal.")
            break
    return

Função 'agente_nutricionista_ludico' (versão com análise de receita) definida com sucesso!


In [13]:
# Célula 4: Menu Principal e Loop de Execução
# --- Função do Menu Principal ---
def rodar_assistente_culinario():
    """
    Exibe o menu principal e gerencia a interação do usuário com os agentes.
    """
    exibir_texto_markdown("# 🤖 Bem-vindo(a) ao Seu Super Assistente Culinário Multiagente! 🤖")

    while True:
        print("\n" + "="*50)
        print("🌟 MENU PRINCIPAL DE AGENTES 🌟")
        print("="*50)
        print("\nCom qual dos nossos especialistas você gostaria de conversar hoje?\n")
        print("1. 🍳 Chef Criativo de Geladeira (Ideias de pratos e receitas)")
        print("2. 💰 Caçador de Ofertas (Melhores preços e gestão de despensa)")
        print("3. 🥗 Nutricionista Lúdico (Dicas de alimentação saudável)")
        print("4. 🚪 Sair da Aplicação")
        print("-"*50)

        escolha_usuario = input("Digite o número da sua escolha: ")

        if escolha_usuario == '1':
            exibir_texto_markdown("\n--- 🧑‍🍳 Conectando com o Chef Criativo... ---")
            agente_chef_criativo()
        elif escolha_usuario == '2':
            exibir_texto_markdown("\n--- 💸 Conectando com o Caçador de Ofertas... ---")
            agente_cacador_de_ofertas()
        elif escolha_usuario == '3':
            exibir_texto_markdown("\n--- 🍎 Conectando com o Nutricionista Lúdico... ---")
            agente_nutricionista_ludico()
        elif escolha_usuario == '4':
            exibir_texto_markdown("\n👋 Obrigado por usar o Assistente Culinário! Esperamos te ver cozinhando conosco novamente em breve! Tchau! 👋")
            break
        else:
            exibir_texto_markdown("\n⚠️ Opção inválida! Por favor, escolha um número válido do menu (1 a 4).")

        if escolha_usuario != '4':
            print("\nRetornando ao menu principal...")

In [ ]:
# Célula 5: Executar a Aplicação
rodar_assistente_culinario()

> # 🤖 Bem-vindo(a) ao Seu Super Assistente Culinário Multiagente! 🤖


🌟 MENU PRINCIPAL DE AGENTES 🌟

Com qual dos nossos especialistas você gostaria de conversar hoje?

1. 🍳 Chef Criativo de Geladeira (Ideias de pratos e receitas)
2. 💰 Caçador de Ofertas (Melhores preços e gestão de despensa)
3. 🥗 Nutricionista Lúdico (Dicas de alimentação saudável)
4. 🚪 Sair da Aplicação
--------------------------------------------------
